![Banner logo](https://raw.githubusercontent.com/CitrineInformatics/community-tools/master/templates/fig/citrine_banner_2.png "Banner logo")

## Visualization in Python

*Authors: Zach del Rosario (zdelrosario@citrine.io)*

### Learning outcomes
By working through this notebook, you will be able to:

* Re-create a number of different plots using `matplotlib`
* Draw conclusions about data using histograms, boxplots, and scatterplots

Tips:
* *Google is your friend*: If you don't know how to do something and can't find it in the examples, *first* try searching for something related on the web. If you can't find it after 5 minutes of searching on your own, please do ask a question. Learning how to search for your own answers is a valuable skill, and you are *expected* to have to look stuff up in this exercise.
* Not all the code here will make sense if you're new to python. That's ok! I've tried to scaffold the exercises to focus on `matplotlib` syntax, rather than more general python syntax.

In [ ]:
# Setup
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from numpy.linalg import svd
# For jupyter-matplotlib compatibility
%matplotlib inline

# External filename handling
dir_incl = "./incl/"
assignment_prefix = "04_"
filename_alloy_data = "./data/wiki_comp.csv"

# Load data
df_data = pd.read_csv(filename_alloy_data)
# Extract the series number
df_data = df_data.assign(
    series=list(map(lambda s: int(s.replace("V-", "")[0]), df_data.alloy))
)


When studying a new dataset, it's always a good idea to do some simple 'first checks'

In [ ]:
# Check the data's shape
df_data.shape


We have 137 observations on 28 variables. Because there are so many variables, these data will be difficult to visualize!

In [ ]:
# Glimpse the data
df_data.head()


This is a dataset about Aluminum alloys, scraped from [Wikipedia](https://en.wikipedia.org/wiki/Aluminium_alloy#Wrought_alloys). The `alloy` column identifies the specific alloy, `al_percent` gives the percent Aluminum content (by weight), `series` gives the alloy series number (defining "families" of alloys), and the remaining columns give other alloying elements. From the "head" of the data (the first few observations) we can see the alloying elements are *sparse* -- many of the entries are zero.

### Learning `matplotlib` syntax by doing

### Q1: Reproduce This Plot
Use `matplotlib` to reproduce the following histogram. Use the [documentation](https://matplotlib.org/3.1.0/gallery/statistics/hist.html) to do so. Make sure to learn how to change *all* of the following:
* Font sizes
* Number of histogram bins
* Axis limits
* Axis labels

In [ ]:
###
# TASK: Reproduce the histogram below
# TODO: Consult the matplotlib documentation to help you re-create the histogram below
###

# -- WRITE YOUR CODE BELOW -----



<img src="./incl/04_q1_target.png">
<!-- task-end -->

### Q2: What does this histogram tell you?
List _at least 3_ observations conclusions about the data you can draw, based on the histogram above.


---

Next, we'll study how alloy composition varies across the various Al series.

In [ ]:
series = list(range(1, 9))
mean_percentages = [np.mean(df_data[df_data.series == s]["al_percent"])
                    for s in series]  # List comprehension
print("Series, Mean Percent")
for ind in range(len(series)):
    print("     {0:},   {1:2.1f}".format(series[ind], mean_percentages[ind]))


This shows that Series 3 and 6 have the highest `Al` content, on average. However, this does not give us as much information as a full histogram. We could plot histograms for the individual series, as demonstrated below:

In [ ]:
series1_al = df_data[df_data.series == 1]["al_percent"]
# Solution code hidden!
# No need to write code here



<img src="./incl/04_series_hist.png">
<!-- task-end -->

However, it is challenging to compare multiple histograms against each other. Instead, we can use *boxplots* to summarize and plot the various series in the same plot.

### Q3: Reproduce This Plot
Use `matplotlib` to reproduce the following [boxplots](https://towardsdatascience.com/understanding-boxplots-5e2df7bcbd51) depicting each aluminum series. Use the [documentation](https://matplotlib.org/3.1.1/gallery/statistics/boxplot_demo.html) to help you do so.

Hints:
* You may re-use lessons you learned from Q1
* plt.boxplot() can take a list of different data sets to produce multiple boxplots on the same figure. You can use a *list comprehension* like in `mean_percentages` above to create such a list, and pass it to plt.boxplot()
* The `df_data[df_data.series == ind]["al_percent"]` pattern above is useful, and may be used in a list comprehension or loop over `ind`.

In [ ]:
###
# TASK: Reproduce the boxplots below
# TODO: Consult the matplotlib documentation to help you re-create the boxplots below
###

# -- WRITE YOUR CODE BELOW -----



<img src="./incl/04_q3_target.png">
<!-- task-end -->

### Q4: What does this plot tell you?
List _at least 3_ observations conclusions about the data you can draw, based on the boxplots above.


---

Studying *all* the variables at once is difficult, as we cannot directly visualize 25-dimensional space! Instead, we will use different *dimension reduction* techniques to visualize the data while preserving low-dimensional structure. We will illustrate two methods:

- Principal Component Analysis [PCA](https://en.wikipedia.org/wiki/Principal_component_analysis) is a classic dimension reduction technique that seeks to preserve *variance* in the original data while reducing the number of dimensions. In this approach, the resulting reduced coordinates have a clear interpretation as *distance along special directions* within the original space.
- [t-SNE](https://lvdmaaten.github.io/tsne/) stands for *t-distributed Stochastic Neighbor Embedding*. It creates a nonlinear dimension reduction that seeks to preserve distances between points. While the resulting coordinates have *no clear interpretation*, t-SNE is highly effective at detecting clusters in data.

In [ ]:
# Prepare data
X_series = df_data["series"]
X_orig = df_data.drop(["alloy", "al_percent", "series"], axis=1).values

# Perform PCA
X_orig = df_data.drop(["alloy", "al_percent", "series"], axis=1).values
U, s, Vh = svd(X_orig)
X_pca = np.dot(X_orig, Vh[:2, :].T)

# Perform t-SNE
# t-SNE is a random algorithm; set seed for reproducibility
np.random.seed(101)
X_tsne = TSNE(
    n_components=2
).fit_transform(X_orig)

print("X_orig.shape = ({})".format(X_orig.shape))
print("X_pca.shape  = ({})".format(X_pca.shape))
print("X_tsne.shape = ({})".format(X_tsne.shape))


We can see the modified datasets `X_pca` and `X_tsne` have just two variables, but the same number of observations as `X_orig`. We can use these two coordinates to plot the data and look for patterns.

---

### PCA

When performing PCA, we need to decide how many principal components ("special directions") in the data to retain. We do this by looking at a plot of the singular values, which describe how much variance is captured along each of the principal directions. This kind of plot is sometimes called a [scree plot](https://en.wikipedia.org/wiki/Scree_plot).

In [ ]:
fig, ax1 = plt.subplots()

# First axis, singular values
ax1.plot(s, 'k.')
ax1.set_xlabel("Index")
ax1.set_ylabel("Singular Value")

ax2 = ax1.twinx()
# Second axis, sum
ax2.plot(np.cumsum(s) / np.sum(s), 'b.--')
ax2.set_ylabel("Total Variance", color="blue")
ax2.tick_params(axis="y", labelcolor="blue")

fig.tight_layout()


The total variance captured in the first two dimensions is less than `50%`; this suggests that a two-dimensional projection of the data will not be very effective. Regardless, let's try plotting the data using just the first two PCA components.

In [ ]:
# CODE REDACTED...
# You'll learn to use plt.scatter() in Q5



<img src="./incl/04_ex_pca.png">
<!-- task-end -->

We can see a bit of clustering in the data -- the `Series 7` alloys are largely separated from other alloys. However, we can find more distinct clusters by using the t-SNE projection.

---

### Q5: Reproduce This t-SNE Plot
Use `matplotlib` to reproduce the following scatterplot on the two t-SNE coordinates in `X_tsne`. The easiest way is probably by using `plt.scatter()`, which is documented [here](https://matplotlib.org/3.1.1/gallery/shapes_and_collections/scatter.html).

Hints:
* You may re-use lessons you learned from Q1 and Q2
* You will need to introduce a colormap to mark the alloys from different series. [This tutorial](https://gist.github.com/jakevdp/8a992f606899ac24b711) may be helpful
* I use the colormap `viridis` below; it is a [perceptually uniform](http://bids.github.io/colormap/) colorscale that avoids introducing erroneous patterns in visualizations

In [ ]:
###
# TASK: Reproduce the scatterplot below
# TODO: Consult the matplotlib documentation to help you re-create the scatterplot below
###

# -- WRITE YOUR CODE BELOW -----



<img src="./incl/04_q5_target.png">
<!-- task-end -->

Interpreting a t-SNE plot is [subtle and challenging](https://distill.pub/2016/misread-tsne/), but also extremely powerful. One of the caveats of t-SNE is that the coordinates themselves do not have a simple interpretation, so the location of clusters in the two coordinates fairly arbitrary. The distances between clusters is also not necessarily meaningful. A well-calibrated t-SNE will tend to reveal clustering in the data, in which case points nearby in t-SNE space can be understood to be close in the original space.

### Q6: What does this plot tell you?
List _at least 3_ observations conclusions about the data you can draw, based on the t-SNE scatterplot above. Use both the observed clusters and the original Series labels in your analysis.
